In [1]:
# imports
import graphlab as gl
import numpy as np
import pandas as pd

In [2]:
# load data
train_df = pd.read_csv('TRAIN.csv', index_col=0)
test_df = pd.read_csv('TEST.csv', index_col = 0)

# fill missing values
train_df['Network type subscription in Month 1'] = train_df['Network type subscription in Month 1'].fillna('3G')
train_df['Network type subscription in Month 2'] = train_df['Network type subscription in Month 2'].fillna('3G')

train_df['Most Loved Competitor network in in Month 1'] = train_df['Most Loved Competitor network in in Month 1'].fillna('Uxaa')
train_df['Most Loved Competitor network in in Month 2'] = train_df['Most Loved Competitor network in in Month 2'].fillna('Uxaa')

# drop empty rows
train_df.dropna(inplace=True, how='any', axis=0)



In [3]:
train_df.dtypes

network_age                                    float64
Customer tenure in month                       float64
Total Spend in Months 1 and 2 of 2017          float64
Total SMS Spend                                float64
Total Data Spend                               float64
Total Data Consumption                         float64
Total Unique Calls                             float64
Total Onnet spend                              float64
Total Offnet spend                             float64
Total Call centre complaint calls              float64
Network type subscription in Month 1            object
Network type subscription in Month 2            object
Most Loved Competitor network in in Month 1     object
Most Loved Competitor network in in Month 2     object
Churn Status                                   float64
dtype: object

In [4]:
# load data
train_sf = gl.SFrame('TRAIN.csv')
test_sf = gl.SFrame('TEST.csv')

train_sf.column_names

This non-commercial license of GraphLab Create for academic use is assigned to johnthas@yahoo.com and will expire on October 31, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1517533290.log


Finished parsing file /Users/robert/Documents/Kaggle/DSN Telecoms Churn 2018/TRAIN.csv

Parsing completed. Parsed 100 lines in 0.053946 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,float,float,float,float,float,int,int,int,int,str,str,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Parsing completed. Parsed 100 lines in 0.024294 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,float,float,float,float,float,int,int,int,int,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/robert/Documents/Kaggle/DSN Telecoms Churn 2018/TEST.csv

Parsing completed. Parsed 600 lines in 0.011736 secs.

<bound method SFrame.column_names of Columns:
	Customer ID	str
	network_age	int
	Customer tenure in month	float
	Total Spend in Months 1 and 2 of 2017	float
	Total SMS Spend	float
	Total Data Spend	float
	Total Data Consumption	float
	Total Unique Calls	int
	Total Onnet spend	int
	Total Offnet spend	int
	Total Call centre complaint calls	int
	Network type subscription in Month 1	str
	Network type subscription in Month 2	str
	Most Loved Competitor network in in Month 1	str
	Most Loved Competitor network in in Month 2	str
	Churn Status	int

Rows: 1401

Data:
+-------------+-------------+--------------------------+-------------------------------+
| Customer ID | network_age | Customer tenure in month | Total Spend in Months 1 an... |
+-------------+-------------+--------------------------+-------------------------------+
|   ADF0039   |     123     |           4.1            |             76.714            |
|   ADF0041   |     1316    |          43.87           |            98.8904      

### churn prediction requires a time series. we don't have this, so consider linear regression.

In [5]:
train_sf = train_sf.fillna('Network type subscription in Month 1', '3G')
train_sf = train_sf.fillna('Network type subscription in Month 2', '3G')

train_sf = train_sf.fillna('Most Loved Competitor network in in Month 1', 'Uxaa')
train_sf = train_sf.fillna('Most Loved Competitor network in in Month 2', 'Uxaa')

train_sf = train_sf.dropna(columns=None, how='any')

In [6]:
# Make a train-test split
train_data, test_data = train_sf.random_split(0.8)

In [7]:
feature_columns = ['network_age', 'Customer tenure in month', 'Total Spend in Months 1 and 2 of 2017', 'Total SMS Spend', 'Total Data Spend', 'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend', 'Total Offnet spend', 'Total Call centre complaint calls', 'Network type subscription in Month 1', 'Network type subscription in Month 2', 'Most Loved Competitor network in in Month 1', 'Most Loved Competitor network in in Month 2']
# Create a model.
model = gl.linear_regression.create(train_data, target='Churn Status',
                                    features = feature_columns)

# Save predictions to an SFrame (class and corresponding class-probabilities)
predictions = model.predict(test_data)

# Evaluate the model and save the results into a dictionary
results = model.evaluate(test_data)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Linear regression:

--------------------------------------------------------

Number of examples          : 1066

Number of features          : 14

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 1.009954     | 1.229114           | 1.122542             | 0.447797      | 0.443217        |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

SUCCESS: Optimal solution found.

In [8]:
results

{'max_error': 1.0815966715735386, 'rmse': 0.48278806601221935}

In [9]:
model['coefficients']

name,index,value,stderr
(intercept),None,0.724984547255,0.0239306026033
network_age,None,-2.11791380664e-05,0.173673192359
Customer tenure in month,None,-0.000258345980519,5.19532532843
Total Spend in Months 1and 2 of 2017 ...,None,-6.39548186861e-05,2.8611838693e-05
Total SMS Spend,None,0.000785222600188,0.000259149515357
Total Data Spend,None,0.000105341346143,8.31335606381e-05
Total Data Consumption,None,-9.52465897806e-09,2.08449671348e-09
Total Unique Calls,None,-8.98182183849e-05,6.70936274159e-05
Total Onnet spend,None,-2.24407881053e-07,9.34901796968e-07
Total Offnet spend,None,3.02999483832e-06,6.57091816783e-07


In [10]:
predictions = model.predict(test_sf)

In [11]:
type(predictions)

graphlab.data_structures.sarray.SArray

In [12]:
type(test_sf)

graphlab.data_structures.sframe.SFrame

In [13]:
test_sf['predictions'] = predictions

In [14]:
test_sf.head()

Customer ID,network_age,Customer tenure in month,Total Spend in Months 1and 2 of 2017 ...,Total SMS Spend,Total Data Spend
ADF0001,1012,33.73333333,1560.1156,14.04,226.3
ADF0002,137,4.566666667,1705.8368,73.63,5.0
ADF0003,202,6.733333333,1720.3008,4.36,245.0
ADF0004,165,5.5,1732.28,0.0,12.5
ADF0005,759,25.3,1873.06,69.57,412.5
ADF0006,129,4.3,1877.26,46.11,343.75
ADF0007,148,4.933333333,2388.8208,17.99,1.25
ADF0008,1144,38.13333333,2702.35,47.84,48.75
ADF0009,137,4.566666667,2782.5572,179.52,315.0
ADF0010,1456,48.53333333,2934.7028,200.37,108.75


In [15]:
test_sf['Churn Status'] = test_sf['predictions'].apply(lambda x: int(x > 0.5))

In [16]:
test_sf.head()

Customer ID,network_age,Customer tenure in month,Total Spend in Months 1and 2 of 2017 ...,Total SMS Spend,Total Data Spend
ADF0001,1012,33.73333333,1560.1156,14.04,226.3
ADF0002,137,4.566666667,1705.8368,73.63,5.0
ADF0003,202,6.733333333,1720.3008,4.36,245.0
ADF0004,165,5.5,1732.28,0.0,12.5
ADF0005,759,25.3,1873.06,69.57,412.5
ADF0006,129,4.3,1877.26,46.11,343.75
ADF0007,148,4.933333333,2388.8208,17.99,1.25
ADF0008,1144,38.13333333,2702.35,47.84,48.75
ADF0009,137,4.566666667,2782.5572,179.52,315.0
ADF0010,1456,48.53333333,2934.7028,200.37,108.75


In [17]:
churn_sf = test_sf[['Customer ID', 'Churn Status']]

In [18]:
churn_sf.export_csv('predictions/turi_linear_regression.csv')

### Submission Accuracy: 0.69444

In [19]:
# investigate coefficients further
model['coefficients'].show()

Canvas is accessible via web browser at the URL: http://localhost:54178/index.html
Opening Canvas in default web browser.


## Play around with features

In [20]:
engineered_columns = ['network_age',  'Total SMS Spend', 'Total Data Spend', 'Total Data Consumption',  'Total Onnet spend', 'Total Offnet spend', 'Total Call centre complaint calls',   'Most Loved Competitor network in in Month 2']
# Create a model.
model = gl.linear_regression.create(train_data, target='Churn Status',
                                    features = engineered_columns)

# Save predictions to an SFrame (class and corresponding class-probabilities)
predictions = model.predict(test_data)

# Evaluate the model and save the results into a dictionary
results = model.evaluate(test_data)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Linear regression:

--------------------------------------------------------

Number of examples          : 1074

Number of features          : 8

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 0.004373     | 1.180853           | 1.146027             | 0.454587      | 0.493790        |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

SUCCESS: Optimal solution found.

In [21]:
results

{'max_error': 1.1131370173312676, 'rmse': 0.49039183440189454}

## Evaluate a Decision Tree

In [22]:
chosen_columns = ['network_age',  'Total SMS Spend', 'Total Data Spend', 'Total Data Consumption',  'Total Onnet spend', 'Total Offnet spend', 'Most Loved Competitor network in in Month 2']
model = gl.decision_tree_classifier.create(train_data, target='Churn Status', features=chosen_columns, max_depth=4)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Decision tree classifier:

--------------------------------------------------------

Number of examples          : 1056

Number of classes           : 2

...

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Validation-accuracy | Training-log_loss | Validation-log_loss |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

| 1         | 0.006487     | 0.756629          | 0.500000            | 0.613111          | 0.675390            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [23]:
predicitons = model.classify(test_data)

In [24]:
results = model.evaluate(test_data)
results

{'accuracy': 0.675, 'auc': 0.7440577031675137, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |   33  |
 |      0       |        0        |   96  |
 |      1       |        0        |   58  |
 |      1       |        1        |   93  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns], 'f1_score': 0.6714801444043322, 'log_loss': 0.6416261692500824, 'precision': 0.7380952380952381, 'recall': 0.6158940397350994, 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-----+-----+
 | threshold | fpr | tpr |  p  |  n  |
 +-----------+-----+-----+-----+-----+
 |    0.0    | 1.0 | 1.0 | 151 | 129 |
 |   1e-05   | 1.0 | 1.0 | 151 | 129 |
 |   2e-05   | 1.0 | 1.0 | 151 | 129 |
 |   3e-05

In [25]:
model.show(view="Tree", tree_id=0)

Canvas is accessible via web browser at the URL: http://localhost:54178/index.html
Opening Canvas in default web browser.


## Try Random Forest

In [26]:
model = gl.random_forest_classifier.create(train_data, target='Churn Status',features=chosen_columns, num_trees=50, max_depth=6)

[WARNING] graphlab.toolkits.classifier.random_forest_classifier: The `num_trees` keyword argument is deprecated. Please use the `max_iterations` argument instead. Any value provided for `num_trees` will be used in place of `max_iterations`.


PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Random forest classifier:

--------------------------------------------------------

Number of examples          : 1072

...

| 4         | 0.025964     | 0.837687          | 0.833333            | 0.445455          | 0.480043            |

| 5         | 0.029136     | 0.826493          | 0.854167            | 0.447729          | 0.476897            |

| 6         | 0.032700     | 0.837687          | 0.833333            | 0.443809          | 0.495311            |

| 11        | 0.068222     | 0.833955          | 0.833333            | 0.440031          | 0.479410            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [27]:
predicitons = model.classify(test_data)

In [28]:
results = model.evaluate(test_data)
results

{'accuracy': 0.6892857142857143,
 'auc': 0.7707787874120843,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |   27  |
 |      0       |        0        |  102  |
 |      1       |        0        |   60  |
 |      1       |        1        |   91  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.6765799256505577,
 'log_loss': 0.5804768589944913,
 'precision': 0.7711864406779662,
 'recall': 0.6026490066225165,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-----+-----+
 | threshold | fpr | tpr |  p  |  n  |
 +-----------+-----+-----+-----+-----+
 |    0.0    | 1.0 | 1.0 | 151 | 129 |
 |   1e-05   | 1.0 | 1.0 | 151 | 129 |
 |   2e-05   | 1.0 | 1.0 | 15

## Try Gradient Boosted Regression Trees

In [53]:
model = gl.boosted_trees_classifier.create(train_data, target='Churn Status', features=chosen_columns,
                                           max_iterations=30,
                                           max_depth = 4)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 1065

Number of classes           : 2

...

| 4         | 0.031367     | 0.785915          | 0.763636            | 0.506745          | 0.538924            |

| 5         | 0.040524     | 0.797183          | 0.727273            | 0.487168          | 0.527743            |

| 6         | 0.044563     | 0.795305          | 0.745455            | 0.470156          | 0.516524            |

| 11        | 0.068321     | 0.830047          | 0.745455            | 0.409225          | 0.497031            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [54]:
results = model.evaluate(test_data)
results

{'accuracy': 0.7071428571428572,
 'auc': 0.7669028184198369,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |   34  |
 |      0       |        0        |   95  |
 |      1       |        0        |   48  |
 |      1       |        1        |  103  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.7152777777777778,
 'log_loss': 0.5992492460548107,
 'precision': 0.7518248175182481,
 'recall': 0.6821192052980133,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-----+-----+
 | threshold | fpr | tpr |  p  |  n  |
 +-----------+-----+-----+-----+-----+
 |    0.0    | 1.0 | 1.0 | 151 | 129 |
 |   1e-05   | 1.0 | 1.0 | 151 | 129 |
 |   2e-05   | 1.0 | 1.0 | 15

In [48]:
predictions = model.classify(test_sf)

In [49]:
test_sf.remove_columns(['predictions', 'Churn Status'])

KeyError: 'Cannot find column predictions'

In [50]:
predictions

class,probability
0,0.616735965014
1,0.801433980465
0,0.562462091446
1,0.73265850544
1,0.727343142033
1,0.819014787674
1,0.854239165783
1,0.845888793468
1,0.782670974731
1,0.851804375648


In [51]:
test_sf['Churn Status'] = predictions['class']

In [52]:
churn_sf = test_sf[['Customer ID', 'Churn Status']]
churn_sf.export_csv('predictions/turi_gradient_boosted.csv')

## Try a Neural Network

In [ ]:
chosen_columns = ['network_age',  'Total SMS Spend', 'Total Data Spend', 'Total Data Consumption',  'Total Onnet spend', 'Total Offnet spend', 'Most Loved Competitor network in in Month 2', 'Churn Status']
data = train_data[chosen_columns]
val = test_data[chosen_columns]
data['loves_Uxaa'] = data['Most Loved Competitor network in in Month 2'] == 'Uxaa'
val['loves_Uxaa'] = val['Most Loved Competitor network in in Month 2'] == 'Uxaa'

In [ ]:
data.column_names

In [ ]:
data.remove_column('Most Loved Competitor network in in Month 2')
val.remove_column('Most Loved Competitor network in in Month 2')
new_columns = ['network_age',  'Total SMS Spend', 'Total Data Spend', 'Total Data Consumption',  'Total Onnet spend', 'Total Offnet spend', 'loves_Uxaa']

In [ ]:
net = gl.deeplearning.create(data, target='Churn Status', features=new_columns, network_type='perceptrons')

In [ ]:
m = gl.neuralnet_classifier.create(data, validation_set=val,
                                         target='Churn Status',
                                         network=net)

In [ ]:
data['features'] = m.extract_features(data)

In [ ]:
pred = m.classify(val)

In [ ]:
pred[:10]

In [ ]:
score = pred[:]['class'] == val[:]['Churn Status']

In [ ]:
print float(sum(score))/len(score)

In [ ]:
model = gl.boosted_trees_classifier.create(data, target='Churn Status', features=['features'],
                                           max_depth = 1)

In [ ]:
predicitons = model.classify(test_data)

In [ ]:
results = model.evaluate(test_data)
results